In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
fpath = './insurance_data.csv'
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate
import pickle
from tqdm import tqdm
import plotly as py
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [137]:
data_orig = pd.read_csv(fpath)
data = data_orig

#0 => Female; 1=> Male
data['ft4'] = data['ft4'].astype('category').cat.codes

#0 => No; 1=> Yes
data['ft5'] = data['ft5'].astype('category').cat.codes

#0 => NE; 1=> NW; 2=> SE; 3=> SW
data['ft6'] = data['ft6'].astype('category').cat.codes

data['ft7'] = data['ft2']*data['ft5']
Xs = data.iloc[:,:6].values
print(Xs.shape)
Xs = np.append(Xs,(Xs[:,1]*Xs[:,4]).reshape((Xs.shape[0],1)), axis=1)
for i in [0,1]:
    for j in [2,3,4]:
        Xs = np.append(Xs,(Xs[:,i]**j).reshape((Xs.shape[0],1)), axis=1)
ys = data.iloc[:,6].values
print(Xs.shape)

   ft1     ft2  ft3     ft4  ft5        ft6      charges
0   19  27.900    0  female  yes  southwest  16884.92400
1   18  33.770    1    male   no  southeast   1725.55230
2   28  33.000    3    male   no  southeast   4449.46200
3   33  22.705    0    male   no  northwest  21984.47061
4   32  28.880    0    male   no  northwest   3866.85520
(1338, 6)
(1338, 13)


In [132]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import SCORERS
from math import sqrt
from sklearn.preprocessing import StandardScaler
def randomForest(model, X, y):
    kf = KFold(n_splits=10)
    rmse_train, rmse_test, oob_error = 0, 0, 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model.fit(X_train, y_train)
        rmse_train += sqrt(mean_squared_error(y_train, model.predict(X_train)))
        rmse_test += sqrt(mean_squared_error(y_test, model.predict(X_test)))
        oob_error += 1 - model.oob_score_
    return rmse_train / 10, rmse_test / 10, oob_error/10

In [54]:
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
rmse_train_arr, rmse_test_arr, oob_arr = [],[],[]
max_depth = 4
trees = list(range(5,200,5))
trees = [600]
for num_feat in tqdm(range(1,7)):
    rmse_train_arr.append([])
    rmse_test_arr.append([])
    oob_arr.append([])
    for num_trees in trees:
        RF = RandomForestRegressor(n_estimators = num_trees, max_depth = max_depth, max_features = num_feat, bootstrap = True, oob_score=True)
        a, b, c = randomForest(RF,Xs,ys)
        rmse_train_arr[num_feat-1].append(a)
        rmse_test_arr[num_feat-1].append(b)
        oob_arr[num_feat-1].append(c)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:37<00:00,  6.45s/it]


In [185]:
X = data.copy()
X = X.drop(['charges'], axis=1)

X.loc[X['ft4'] == 'male', 'ft4'] = 0
X.loc[X['ft4'] == 'female', 'ft4'] = 1

X.loc[X['ft5'] == 'no', 'ft5'] = 0
X.loc[X['ft5'] == 'yes', 'ft5'] = 1

X.loc[X['ft6'] == 'southwest', 'ft6'] = 0
X.loc[X['ft6'] == 'southeast', 'ft6'] = 1
X.loc[X['ft6'] == 'northwest', 'ft6'] = 2
X.loc[X['ft6'] == 'northeast', 'ft6'] = 3

In [194]:
best_depth = 6
best_num_trees = 200
best_feat = 7
#for best_depth in range(4,8):
best_RF = RandomForestRegressor(n_estimators = best_num_trees, max_depth = best_depth,  min_samples_split=10,
                                 random_state=0, bootstrap = True, oob_score=True)
X_train, X_test, y_train, y_test = train_test_split(X, ys, test_size=0.1, random_state=0)
best_RF.fit(X_train, y_train)
y_pred = best_RF.predict(X_test)
print(sqrt(mse(y_test,y_pred)))

3999.055078308132


In [192]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
import pickle

params = {'hidden_layer_sizes': [(60, 20), (50, 20), 
                                 (60, 30), (50, 30),
                                 (60, 40), (60, 40)],
          'learning_rate_init': [1e-5, 1e-6, 1e-7, 1e-8], 
          'alpha': [0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4]}

X_train, X_test, y_train, y_test = train_test_split(X, ys, test_size=0.1, random_state=0)
nn = MLPRegressor(activation='relu')
clf = GridSearchCV(nn, params, n_jobs=-1, cv=5, verbose = True)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
#X_train_scaled = np.load('X_train')
#X_test_scaled = np.load('X_test','rb')
clf = clf.fit(X_train_scaled, y_train)
X_test_scaled = scaler.transform(X_test)
y_pred = clf.predict(X_test_scaled)
print(sqrt(mse(y_pred, y_test)))

Fitting 5 folds for each of 168 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed:  6.4min finished


20915.617373012563


In [187]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error as mse
gb = GradientBoostingRegressor(random_state=42)
params={'n_estimators':list(range(1000,1500,50)),
          'learning_rate':[1e-8, 1e-6, 1e-5,1e-4,1e-2] }
X_train, X_test, y_train, y_test = train_test_split(X, ys, train_size=0.9)
gs = GridSearchCV(gb, params, n_jobs=-1, cv=5, verbose=True)
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)
print(sqrt(mse(y_test,y_pred)))

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   37.3s finished


3710.376993517298


In [179]:
print(gs.best_estimator_)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=3,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1450,
             n_iter_no_change=None, presort='auto', random_state=42,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)
